In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})

from glob import glob

dirs = sorted(os.listdir("outs"))
list(enumerate(dirs))

[(0, '181001_dense169_3D_bac18'),
 (1, '181002_dense201_3D_bac18'),
 (2, '181003_dense169_shape'),
 (3, 'sdf'),
 (4, 'sdif')]

In [3]:
target_dir =  os.getcwd() + "/outs/" + dirs[0]
print(target_dir)

/data2/DW/180930_bac/bacteria_classification/outs/181001_dense169_3D_bac18


In [4]:
import torch; torch_device = torch.device("cpu")
from models.Densenet3d_nomaxpool import densenet169, densenet121, densenet201
net = densenet169(num_classes=18, sample_size=64, sample_duration=96)
net = torch.nn.DataParallel(net)
state = torch.load(sorted(glob(target_dir + "/*.tar"))[-1])["network"]
net.load_state_dict(state)
net = net.module
net.to(torch_device)
print("Net Load")

Net Load


In [5]:
from datas.BacLoader import bacLoader
import datas.preprocess3d as preprocess

data_path = "/data2/DW/180930_bac/Bacteria/"
test_preprocess = preprocess.get_preprocess("test")
test_loader = bacLoader(data_path + "test", 1, task="multi",
                             transform=test_preprocess,
                             num_workers=16, infer=True, shuffle=False, drop_last=False)
len(test_loader)

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


337

In [ ]:
import torch.nn.functional as F
torch.set_printoptions(precision=5)

imgs = []
net.eval()
with torch.no_grad():
    idx_to_class = {v:k for k, v in test_loader.dataset.class_to_idx.items()}
    
    for input_, target_, path in test_loader:
        input_ = input_.to(torch_device)
        output_ = net(input_)
        output_ = F.softmax(output_, dim=1)
        _, idx = output_.max(dim=1)
        
        if idx.item() != target_.item():
            topk, indices = output_.topk(3)            
            label = idx_to_class[target_.item()]
            indices = [(idx_to_class[i], v) for i, v in zip(indices.view(-1).tolist(), topk.view(-1).tolist())]
            title = "Label : %s | Pred : %s : %.4f, %s : %.4f, %s : %.4f"%(label,
                                                                                                           *[z for i in indices for z in i])
            imgs.append((path, title))

In [ ]:
len(imgs)

In [ ]:
from scipy.io import loadmat
for path, title in imgs:
    pass